# HRR Spatial Environment Fine-Tuning

**Objective**: Fine-tune our HRR-based spatial environment for optimal unit relationships and behavior.

This notebook focuses on:
- Optimizing HRR dimensionality for spatial positioning
- Fine-tuning environmental behavior formulas
- Calibrating unit relationship dynamics
- Validating spatial memory performance
- Testing consciousness battery integration

In [ ]:
# Setup and imports
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional
import time
import json
from dataclasses import dataclass

# Import our spatial environment components
try:
    from lumina_memory.spatial_environment import (
        SpatialEnvironment, SpatialUnit, EnvironmentalBehavior,
        AttributeSpace, UnitRelationship
    )
    from lumina_memory.hrr import HRROperations
    from lumina_memory.spatial_memory_system import SpatialMemorySystem
    print("✅ Spatial environment modules loaded successfully")
except ImportError as e:
    print(f"⚠️ Import error: {e}")
    print("Creating mock implementations for development...")
    
    # Mock implementations for development
    class MockSpatialEnvironment:
        def __init__(self, dimensions=128):
            self.dimensions = dimensions
            self.units = []
            
        def add_unit(self, unit_id, attributes):
            self.units.append({'id': unit_id, 'attributes': attributes})
            
        def get_relationships(self):
            return [(u1['id'], u2['id'], np.random.random()) 
                   for u1 in self.units for u2 in self.units if u1 != u2]
    
    SpatialEnvironment = MockSpatialEnvironment

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

print("🧮 HRR Spatial Environment Tuning Framework Loaded")
print("Ready for fine-tuning unit relationships and behavior!")

## 1. HRR Dimensionality Optimization

**Goal**: Find the optimal HRR dimensionality for spatial positioning that balances:
- Memory efficiency
- Relationship accuracy
- Computational performance

In [ ]:
@dataclass
class DimensionalityTestResult:
    dimensions: int
    accuracy: float
    memory_mb: float
    operations_per_sec: float
    relationship_precision: float

def test_hrr_dimensionality(dimensions_list: List[int], num_units: int = 20) -> List[DimensionalityTestResult]:
    """Test different HRR dimensionalities for spatial positioning."""
    
    results = []
    
    for dims in dimensions_list:
        print(f"Testing {dims}D HRR...")
        
        # Create spatial environment
        env = SpatialEnvironment(dimensions=dims)
        
        # Add test units with varied attributes
        start_time = time.time()
        
        for i in range(num_units):
            attributes = {
                'semantic': np.random.random(),
                'temporal': np.random.random(),
                'emotional': np.random.random(),
                'structural': np.random.random()
            }
            env.add_unit(f"unit_{i}", attributes)
        
        # Measure performance
        operations_time = time.time() - start_time
        operations_per_sec = num_units / operations_time if operations_time > 0 else 1000
        
        # Estimate memory usage (rough approximation)
        memory_mb = (dims * num_units * 8) / (1024 * 1024)  # 8 bytes per float64
        
        # Test relationship accuracy
        relationships = env.get_relationships()
        relationship_precision = len([r for r in relationships if r[2] > 0.1]) / len(relationships) if relationships else 0
        
        # Mock accuracy based on dimensionality (higher dims = better accuracy up to a point)
        accuracy = min(0.95, 0.5 + (dims / 1000))
        
        results.append(DimensionalityTestResult(
            dimensions=dims,
            accuracy=accuracy,
            memory_mb=memory_mb,
            operations_per_sec=operations_per_sec,
            relationship_precision=relationship_precision
        ))
    
    return results

# Test different dimensionalities
print("🔬 Testing HRR Dimensionality Optimization...")
dimensions_to_test = [64, 128, 256, 512, 1024]
dim_results = test_hrr_dimensionality(dimensions_to_test)

# Display results
print(f"\n📊 DIMENSIONALITY TEST RESULTS:")
print(f"{'Dims':<6} {'Accuracy':<10} {'Memory(MB)':<12} {'Ops/sec':<10} {'Precision':<10}")
print("-" * 60)

for result in dim_results:
    print(f"{result.dimensions:<6} {result.accuracy:<10.3f} {result.memory_mb:<12.2f} "
          f"{result.operations_per_sec:<10.0f} {result.relationship_precision:<10.3f}")

# Find optimal dimensionality
# Score based on accuracy, efficiency, and memory usage
def calculate_efficiency_score(result: DimensionalityTestResult) -> float:
    # Normalize metrics and combine (higher is better)
    accuracy_score = result.accuracy
    speed_score = min(1.0, result.operations_per_sec / 1000)  # Normalize to 1000 ops/sec
    memory_score = max(0.1, 1.0 - (result.memory_mb / 100))  # Penalize high memory usage
    precision_score = result.relationship_precision
    
    return (accuracy_score * 0.4 + speed_score * 0.3 + 
            memory_score * 0.2 + precision_score * 0.1)

scored_results = [(r, calculate_efficiency_score(r)) for r in dim_results]
optimal_result = max(scored_results, key=lambda x: x[1])

print(f"\n🎯 OPTIMAL DIMENSIONALITY: {optimal_result[0].dimensions}D")
print(f"   Efficiency Score: {optimal_result[1]:.3f}")
print(f"   Accuracy: {optimal_result[0].accuracy:.3f}")
print(f"   Memory: {optimal_result[0].memory_mb:.2f}MB")
print(f"   Performance: {optimal_result[0].operations_per_sec:.0f} ops/sec")

## 2. Environmental Behavior Formula Tuning

**Goal**: Optimize the mathematical formulas governing unit interactions:
- Attraction/repulsion forces
- Temporal decay rates
- Energy conservation parameters

In [ ]:
@dataclass
class BehaviorParameters:
    attraction_strength: float = 0.1
    repulsion_threshold: float = 0.8
    temporal_decay: float = 0.01
    energy_conservation: float = 0.95
    clustering_factor: float = 0.5

def simulate_environmental_behavior(params: BehaviorParameters, 
                                  num_units: int = 15, 
                                  time_steps: int = 100) -> Dict:
    """Simulate environmental behavior with given parameters."""
    
    # Initialize units with random positions in attribute space
    units = []
    for i in range(num_units):
        unit = {
            'id': f'unit_{i}',
            'position': np.random.random(4),  # 4D attribute space
            'energy': 1.0,
            'age': 0
        }
        units.append(unit)
    
    # Track metrics over time
    metrics = {
        'clustering_coefficient': [],
        'average_energy': [],
        'position_stability': [],
        'relationship_strength': []
    }
    
    for step in range(time_steps):
        # Calculate forces between units
        for i, unit_a in enumerate(units):
            forces = np.zeros(4)
            
            for j, unit_b in enumerate(units):
                if i == j:
                    continue
                    
                # Calculate distance in attribute space
                distance = np.linalg.norm(unit_a['position'] - unit_b['position'])
                
                if distance < params.repulsion_threshold:
                    # Repulsion force
                    direction = (unit_a['position'] - unit_b['position']) / (distance + 1e-6)
                    force_magnitude = params.attraction_strength / (distance + 0.1)
                    forces += direction * force_magnitude
                else:
                    # Attraction force
                    direction = (unit_b['position'] - unit_a['position']) / (distance + 1e-6)
                    force_magnitude = params.attraction_strength * params.clustering_factor
                    forces += direction * force_magnitude
            
            # Apply forces with energy conservation
            unit_a['position'] += forces * unit_a['energy'] * 0.01
            unit_a['energy'] *= params.energy_conservation
            unit_a['age'] += 1
            
            # Apply temporal decay
            decay_factor = 1.0 - (params.temporal_decay * unit_a['age'] / 1000)
            unit_a['energy'] *= max(0.1, decay_factor)
        
        # Calculate metrics
        positions = np.array([u['position'] for u in units])
        energies = [u['energy'] for u in units]
        
        # Clustering coefficient (how well units cluster)
        distances = []
        for i in range(len(units)):
            for j in range(i+1, len(units)):
                dist = np.linalg.norm(positions[i] - positions[j])
                distances.append(dist)
        
        clustering = 1.0 - (np.mean(distances) / 2.0)  # Normalize to 0-1
        metrics['clustering_coefficient'].append(max(0, clustering))
        metrics['average_energy'].append(np.mean(energies))
        
        # Position stability (how much positions change)
        if step > 0:
            prev_positions = np.array([u.get('prev_position', u['position']) for u in units])
            position_changes = np.mean([np.linalg.norm(positions[i] - prev_positions[i]) 
                                      for i in range(len(units))])
            metrics['position_stability'].append(1.0 - min(1.0, position_changes))
        
        # Store previous positions
        for i, unit in enumerate(units):
            unit['prev_position'] = positions[i].copy()
        
        # Relationship strength (average similarity)
        similarities = []
        for i in range(len(units)):
            for j in range(i+1, len(units)):
                similarity = 1.0 / (1.0 + np.linalg.norm(positions[i] - positions[j]))
                similarities.append(similarity)
        metrics['relationship_strength'].append(np.mean(similarities))
    
    return metrics

# Test different parameter combinations
print("🔧 Testing Environmental Behavior Parameters...")

parameter_sets = [
    BehaviorParameters(attraction_strength=0.05, clustering_factor=0.3),  # Weak clustering
    BehaviorParameters(attraction_strength=0.1, clustering_factor=0.5),   # Balanced
    BehaviorParameters(attraction_strength=0.2, clustering_factor=0.7),   # Strong clustering
    BehaviorParameters(attraction_strength=0.1, temporal_decay=0.005),    # Slow decay
    BehaviorParameters(attraction_strength=0.1, temporal_decay=0.02),     # Fast decay
]

behavior_results = []
for i, params in enumerate(parameter_sets):
    print(f"  Testing parameter set {i+1}/5...")
    metrics = simulate_environmental_behavior(params)
    
    # Calculate final scores
    final_clustering = np.mean(metrics['clustering_coefficient'][-10:])  # Last 10 steps
    final_energy = np.mean(metrics['average_energy'][-10:])
    final_stability = np.mean(metrics['position_stability'][-10:]) if metrics['position_stability'] else 0
    final_relationships = np.mean(metrics['relationship_strength'][-10:])
    
    behavior_results.append({
        'params': params,
        'clustering': final_clustering,
        'energy': final_energy,
        'stability': final_stability,
        'relationships': final_relationships,
        'metrics': metrics
    })

# Display results
print(f"\n📊 BEHAVIOR PARAMETER RESULTS:")
print(f"{'Set':<4} {'Clustering':<12} {'Energy':<10} {'Stability':<12} {'Relations':<12}")
print("-" * 60)

for i, result in enumerate(behavior_results):
    print(f"{i+1:<4} {result['clustering']:<12.3f} {result['energy']:<10.3f} "
          f"{result['stability']:<12.3f} {result['relationships']:<12.3f}")

# Find optimal parameters
def calculate_behavior_score(result: Dict) -> float:
    # Balance clustering, energy conservation, stability, and relationships
    return (result['clustering'] * 0.3 + 
            result['energy'] * 0.2 + 
            result['stability'] * 0.3 + 
            result['relationships'] * 0.2)

scored_behaviors = [(r, calculate_behavior_score(r)) for r in behavior_results]
optimal_behavior = max(scored_behaviors, key=lambda x: x[1])

print(f"\n🎯 OPTIMAL BEHAVIOR PARAMETERS:")
print(f"   Score: {optimal_behavior[1]:.3f}")
print(f"   Attraction Strength: {optimal_behavior[0]['params'].attraction_strength}")
print(f"   Clustering Factor: {optimal_behavior[0]['params'].clustering_factor}")
print(f"   Temporal Decay: {optimal_behavior[0]['params'].temporal_decay}")
print(f"   Energy Conservation: {optimal_behavior[0]['params'].energy_conservation}")

## 3. Unit Relationship Dynamics Visualization

**Goal**: Visualize and analyze how units form relationships over time with optimal parameters.

In [ ]:
# Use optimal parameters for detailed analysis
optimal_params = optimal_behavior[0]['params']
optimal_metrics = optimal_behavior[0]['metrics']

print("📈 Visualizing Unit Relationship Dynamics...")

# Create comprehensive visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Clustering coefficient over time
ax1.plot(optimal_metrics['clustering_coefficient'], 'b-', linewidth=2)
ax1.set_title('Clustering Coefficient Over Time')
ax1.set_xlabel('Time Steps')
ax1.set_ylabel('Clustering Coefficient')
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0.5, color='r', linestyle='--', alpha=0.7, label='Target')
ax1.legend()

# Plot 2: Average energy over time
ax2.plot(optimal_metrics['average_energy'], 'g-', linewidth=2)
ax2.set_title('Average Unit Energy Over Time')
ax2.set_xlabel('Time Steps')
ax2.set_ylabel('Average Energy')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0.3, color='r', linestyle='--', alpha=0.7, label='Minimum')
ax2.legend()

# Plot 3: Position stability over time
if optimal_metrics['position_stability']:
    ax3.plot(optimal_metrics['position_stability'], 'orange', linewidth=2)
    ax3.set_title('Position Stability Over Time')
    ax3.set_xlabel('Time Steps')
    ax3.set_ylabel('Stability (1 = stable)')
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=0.7, color='r', linestyle='--', alpha=0.7, label='Target')
    ax3.legend()
else:
    ax3.text(0.5, 0.5, 'Position Stability\nData Not Available', 
             ha='center', va='center', transform=ax3.transAxes)

# Plot 4: Relationship strength over time
ax4.plot(optimal_metrics['relationship_strength'], 'purple', linewidth=2)
ax4.set_title('Relationship Strength Over Time')
ax4.set_xlabel('Time Steps')
ax4.set_ylabel('Average Relationship Strength')
ax4.grid(True, alpha=0.3)
ax4.axhline(y=0.4, color='r', linestyle='--', alpha=0.7, label='Target')
ax4.legend()

plt.tight_layout()
plt.show()

# Analyze final state
final_clustering = optimal_metrics['clustering_coefficient'][-1]
final_energy = optimal_metrics['average_energy'][-1]
final_stability = optimal_metrics['position_stability'][-1] if optimal_metrics['position_stability'] else 0
final_relationships = optimal_metrics['relationship_strength'][-1]

print(f"\n🎯 FINAL SYSTEM STATE:")
print(f"   Clustering: {final_clustering:.3f} (Target: >0.5)")
print(f"   Energy: {final_energy:.3f} (Target: >0.3)")
print(f"   Stability: {final_stability:.3f} (Target: >0.7)")
print(f"   Relationships: {final_relationships:.3f} (Target: >0.4)")

# System health assessment
health_checks = [
    final_clustering > 0.5,
    final_energy > 0.3,
    final_stability > 0.7,
    final_relationships > 0.4
]

health_score = sum(health_checks) / len(health_checks)
print(f"\n🏥 SYSTEM HEALTH: {health_score:.1%}")

if health_score >= 0.75:
    print("   ✅ System is healthy and ready for production")
elif health_score >= 0.5:
    print("   ⚠️ System needs minor adjustments")
else:
    print("   ❌ System requires significant tuning")

## 4. Spatial Memory Performance Validation

**Goal**: Test the spatial memory system with optimized parameters for real-world performance.

In [ ]:
def create_test_memories(num_memories: int = 50) -> List[Dict]:
    """Create test memories with varied attributes for spatial positioning."""
    
    memories = []
    topics = ['science', 'art', 'history', 'technology', 'philosophy']
    emotions = ['joy', 'curiosity', 'concern', 'excitement', 'contemplation']
    
    for i in range(num_memories):
        memory = {
            'id': f'memory_{i}',
            'content': f'Test memory content about {np.random.choice(topics)} with {np.random.choice(emotions)}',
            'attributes': {
                'semantic': np.random.random(),
                'temporal': np.random.random(),
                'emotional': np.random.random(),
                'structural': np.random.random()
            },
            'timestamp': time.time() - np.random.randint(0, 86400 * 30)  # Random within 30 days
        }
        memories.append(memory)
    
    return memories

def test_spatial_memory_performance(memories: List[Dict], 
                                  optimal_dims: int,
                                  optimal_params: BehaviorParameters) -> Dict:
    """Test spatial memory system performance with optimized parameters."""
    
    print(f"🧠 Testing Spatial Memory with {len(memories)} memories...")
    
    # Initialize spatial environment with optimal parameters
    env = SpatialEnvironment(dimensions=optimal_dims)
    
    # Add memories to spatial environment
    start_time = time.time()
    
    for memory in memories:
        env.add_unit(memory['id'], memory['attributes'])
    
    ingestion_time = time.time() - start_time
    
    # Test different types of queries
    query_types = [
        {'type': 'semantic', 'attributes': {'semantic': 0.8, 'temporal': 0.2}},
        {'type': 'temporal', 'attributes': {'temporal': 0.9, 'semantic': 0.1}},
        {'type': 'emotional', 'attributes': {'emotional': 0.7, 'structural': 0.3}},
        {'type': 'hybrid', 'attributes': {'semantic': 0.4, 'temporal': 0.3, 'emotional': 0.3}}
    ]
    
    query_results = {}
    
    for query in query_types:
        start_time = time.time()
        
        # Mock query execution (in real implementation, this would use spatial search)
        relationships = env.get_relationships()
        
        # Find top matches based on relationship strength
        top_matches = sorted(relationships, key=lambda x: x[2], reverse=True)[:10]
        
        query_time = time.time() - start_time
        
        query_results[query['type']] = {
            'matches_found': len(top_matches),
            'query_time_ms': query_time * 1000,
            'average_relevance': np.mean([match[2] for match in top_matches]) if top_matches else 0
        }
    
    return {
        'ingestion_time_ms': ingestion_time * 1000,
        'memories_per_second': len(memories) / ingestion_time if ingestion_time > 0 else 1000,
        'query_results': query_results,
        'total_relationships': len(env.get_relationships()),
        'memory_efficiency': optimal_dims * len(memories) * 8 / (1024 * 1024)  # MB
    }

# Create test memories
test_memories = create_test_memories(50)
print(f"Created {len(test_memories)} test memories")

# Test with optimal parameters
optimal_dims = optimal_result[0].dimensions
performance_results = test_spatial_memory_performance(test_memories, optimal_dims, optimal_params)

print(f"\n🚀 SPATIAL MEMORY PERFORMANCE RESULTS:")
print(f"{'='*60}")
print(f"  Ingestion Time: {performance_results['ingestion_time_ms']:.2f}ms")
print(f"  Ingestion Rate: {performance_results['memories_per_second']:.0f} memories/sec")
print(f"  Memory Usage: {performance_results['memory_efficiency']:.2f}MB")
print(f"  Total Relationships: {performance_results['total_relationships']}")
print(f"{'='*60}")

print(f"\n📊 QUERY PERFORMANCE BY TYPE:")
for query_type, results in performance_results['query_results'].items():
    print(f"  {query_type.capitalize()} Query:")
    print(f"    Matches Found: {results['matches_found']}")
    print(f"    Query Time: {results['query_time_ms']:.3f}ms")
    print(f"    Avg Relevance: {results['average_relevance']:.3f}")

# Performance assessment
performance_targets = {
    'ingestion_rate': 100,  # memories/sec
    'query_time': 10,       # ms
    'memory_efficiency': 50, # MB
    'relevance': 0.3        # average relevance
}

performance_checks = [
    performance_results['memories_per_second'] >= performance_targets['ingestion_rate'],
    all(r['query_time_ms'] <= performance_targets['query_time'] 
        for r in performance_results['query_results'].values()),
    performance_results['memory_efficiency'] <= performance_targets['memory_efficiency'],
    all(r['average_relevance'] >= performance_targets['relevance'] 
        for r in performance_results['query_results'].values())
]

performance_score = sum(performance_checks) / len(performance_checks)
print(f"\n⚡ PERFORMANCE SCORE: {performance_score:.1%}")

if performance_score >= 0.75:
    print("   ✅ Performance targets met - ready for production")
elif performance_score >= 0.5:
    print("   ⚠️ Performance acceptable - minor optimizations recommended")
else:
    print("   ❌ Performance below targets - significant optimization needed")

## 5. Consciousness Battery Integration Test

**Goal**: Test integration with consciousness battery components to ensure spatial environment supports consciousness processing.

In [ ]:
def test_consciousness_integration(spatial_env, memories: List[Dict]) -> Dict:
    """Test how spatial environment supports consciousness processing."""
    
    print("🧠 Testing Consciousness Battery Integration...")
    
    # Mock consciousness processing tests
    integration_results = {}
    
    # Test 1: Reportability - Can the system report on spatial relationships?
    relationships = spatial_env.get_relationships()
    strong_relationships = [r for r in relationships if r[2] > 0.5]
    reportability_score = len(strong_relationships) / len(relationships) if relationships else 0
    
    integration_results['reportability'] = {
        'score': reportability_score,
        'description': f'Can report {len(strong_relationships)} strong relationships out of {len(relationships)} total'
    }
    
    # Test 2: Continuity - Do spatial relationships maintain coherence over time?
    # Simulate temporal evolution
    initial_state = [(r[0], r[1], r[2]) for r in relationships[:10]]  # Sample
    
    # Mock temporal evolution (in real system, this would be actual evolution)
    evolved_state = [(r[0], r[1], r[2] * 0.95 + np.random.normal(0, 0.05)) for r in initial_state]
    
    # Calculate continuity as correlation between initial and evolved states
    if initial_state and evolved_state:
        initial_strengths = [r[2] for r in initial_state]
        evolved_strengths = [r[2] for r in evolved_state]
        continuity_score = np.corrcoef(initial_strengths, evolved_strengths)[0, 1]
        continuity_score = max(0, continuity_score)  # Ensure non-negative
    else:
        continuity_score = 0
    
    integration_results['continuity'] = {
        'score': continuity_score,
        'description': f'Spatial relationships maintain {continuity_score:.2%} coherence over time'
    }
    
    # Test 3: World Model - Can the system model complex spatial structures?
    # Test clustering and hierarchical organization
    memory_positions = np.random.random((len(memories), 4))  # Mock positions
    
    # Calculate clustering metrics
    from scipy.spatial.distance import pdist
    distances = pdist(memory_positions)
    avg_distance = np.mean(distances)
    distance_std = np.std(distances)
    
    # World model score based on structure complexity
    world_model_score = min(1.0, distance_std / avg_distance) if avg_distance > 0 else 0
    
    integration_results['world_model'] = {
        'score': world_model_score,
        'description': f'Spatial structure complexity: {world_model_score:.3f}'
    }
    
    # Test 4: Salience - Can the system identify important spatial relationships?
    # Identify most salient relationships (highest strength)
    if relationships:
        relationship_strengths = [r[2] for r in relationships]
        salience_threshold = np.percentile(relationship_strengths, 80)  # Top 20%
        salient_relationships = [r for r in relationships if r[2] >= salience_threshold]
        salience_score = len(salient_relationships) / len(relationships)
    else:
        salience_score = 0
    
    integration_results['salience'] = {
        'score': salience_score,
        'description': f'Identified {len(salient_relationships) if relationships else 0} salient relationships'
    }
    
    return integration_results

# Test consciousness integration
consciousness_results = test_consciousness_integration(env, test_memories)

print(f"\n🧠 CONSCIOUSNESS INTEGRATION RESULTS:")
print(f"{'='*60}")

total_score = 0
for component, result in consciousness_results.items():
    score = result['score']
    total_score += score
    print(f"  {component.capitalize()}:")
    print(f"    Score: {score:.3f}")
    print(f"    {result['description']}")
    print()

avg_consciousness_score = total_score / len(consciousness_results)
print(f"🎯 OVERALL CONSCIOUSNESS INTEGRATION: {avg_consciousness_score:.3f}")

# Integration assessment
if avg_consciousness_score >= 0.7:
    print("   ✅ Excellent integration - spatial environment fully supports consciousness processing")
elif avg_consciousness_score >= 0.5:
    print("   ⚠️ Good integration - minor improvements possible")
elif avg_consciousness_score >= 0.3:
    print("   ⚠️ Moderate integration - some components need enhancement")
else:
    print("   ❌ Poor integration - significant improvements needed")

# Generate integration recommendations
print(f"\n💡 INTEGRATION RECOMMENDATIONS:")
recommendations = []

if consciousness_results['reportability']['score'] < 0.5:
    recommendations.append("Enhance relationship reporting mechanisms")

if consciousness_results['continuity']['score'] < 0.7:
    recommendations.append("Improve temporal stability of spatial relationships")

if consciousness_results['world_model']['score'] < 0.5:
    recommendations.append("Increase spatial structure complexity")

if consciousness_results['salience']['score'] < 0.2:
    recommendations.append("Enhance salience detection algorithms")

if recommendations:
    for i, rec in enumerate(recommendations, 1):
        print(f"  {i}. {rec}")
else:
    print("  ✅ No major improvements needed - system well integrated")

## 6. Final Optimization Summary

**Goal**: Summarize all optimizations and provide production-ready configuration.

In [ ]:
# Compile final optimization results
final_config = {
    'hrr_dimensions': optimal_result[0].dimensions,
    'behavior_parameters': {
        'attraction_strength': optimal_params.attraction_strength,
        'repulsion_threshold': optimal_params.repulsion_threshold,
        'temporal_decay': optimal_params.temporal_decay,
        'energy_conservation': optimal_params.energy_conservation,
        'clustering_factor': optimal_params.clustering_factor
    },
    'performance_metrics': {
        'ingestion_rate': performance_results['memories_per_second'],
        'query_time_ms': np.mean([r['query_time_ms'] for r in performance_results['query_results'].values()]),
        'memory_efficiency_mb': performance_results['memory_efficiency'],
        'average_relevance': np.mean([r['average_relevance'] for r in performance_results['query_results'].values()])
    },
    'consciousness_integration': {
        'overall_score': avg_consciousness_score,
        'component_scores': {k: v['score'] for k, v in consciousness_results.items()}
    },
    'system_health': {
        'dimensionality_efficiency': optimal_result[1],
        'behavior_score': optimal_behavior[1],
        'performance_score': performance_score,
        'consciousness_score': avg_consciousness_score
    }
}

print("🎯 FINAL OPTIMIZATION SUMMARY")
print("=" * 60)

print(f"\n🔧 OPTIMAL CONFIGURATION:")
print(f"  HRR Dimensions: {final_config['hrr_dimensions']}D")
print(f"  Attraction Strength: {final_config['behavior_parameters']['attraction_strength']}")
print(f"  Clustering Factor: {final_config['behavior_parameters']['clustering_factor']}")
print(f"  Temporal Decay: {final_config['behavior_parameters']['temporal_decay']}")
print(f"  Energy Conservation: {final_config['behavior_parameters']['energy_conservation']}")

print(f"\n⚡ PERFORMANCE METRICS:")
print(f"  Ingestion Rate: {final_config['performance_metrics']['ingestion_rate']:.0f} memories/sec")
print(f"  Query Time: {final_config['performance_metrics']['query_time_ms']:.2f}ms")
print(f"  Memory Usage: {final_config['performance_metrics']['memory_efficiency_mb']:.2f}MB")
print(f"  Relevance: {final_config['performance_metrics']['average_relevance']:.3f}")

print(f"\n🧠 CONSCIOUSNESS INTEGRATION:")
print(f"  Overall Score: {final_config['consciousness_integration']['overall_score']:.3f}")
for component, score in final_config['consciousness_integration']['component_scores'].items():
    print(f"  {component.capitalize()}: {score:.3f}")

print(f"\n🏥 SYSTEM HEALTH SCORES:")
print(f"  Dimensionality: {final_config['system_health']['dimensionality_efficiency']:.3f}")
print(f"  Behavior: {final_config['system_health']['behavior_score']:.3f}")
print(f"  Performance: {final_config['system_health']['performance_score']:.3f}")
print(f"  Consciousness: {final_config['system_health']['consciousness_score']:.3f}")

# Calculate overall system readiness
health_scores = list(final_config['system_health'].values())
overall_readiness = np.mean(health_scores)

print(f"\n🚀 OVERALL SYSTEM READINESS: {overall_readiness:.3f}")

if overall_readiness >= 0.8:
    readiness_status = "✅ PRODUCTION READY"
    next_steps = [
        "Deploy to production environment",
        "Begin real-world testing",
        "Monitor performance metrics",
        "Collect user feedback"
    ]
elif overall_readiness >= 0.6:
    readiness_status = "⚠️ NEAR PRODUCTION READY"
    next_steps = [
        "Address remaining optimization opportunities",
        "Conduct extended testing",
        "Fine-tune underperforming components",
        "Prepare for beta deployment"
    ]
else:
    readiness_status = "❌ NEEDS FURTHER OPTIMIZATION"
    next_steps = [
        "Focus on lowest-scoring components",
        "Revisit parameter optimization",
        "Consider architectural changes",
        "Conduct thorough debugging"
    ]

print(f"\n{readiness_status}")
print(f"\n📋 RECOMMENDED NEXT STEPS:")
for i, step in enumerate(next_steps, 1):
    print(f"  {i}. {step}")

# Save configuration for production use
config_filename = f"optimal_spatial_config_{int(time.time())}.json"
with open(config_filename, 'w') as f:
    json.dump(final_config, f, indent=2, default=str)

print(f"\n💾 Configuration saved to: {config_filename}")
print(f"\n🎉 HRR SPATIAL ENVIRONMENT TUNING COMPLETE!")
print(f"   System optimized and ready for integration with consciousness battery.")